# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
# !gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# Unzip the dataset.
# This may take some time.
# !unzip -q food-11.zip

## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [3]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# This is for the progress bar.
from tqdm.notebook import tqdm

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [4]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    # transforms.Resize(256),
    transforms.RandomRotation(180),
    transforms.RandomResizedCrop(224),
    # transforms.RandomCrop(224),
    # transforms.CenterCrop(224),
    # transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=.2, hue=0.08),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply(
                [transforms.ColorJitter(brightness=0.4, contrast=0.4,saturation=0.2, hue=0.1)],
                p=0.8
    ),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=9, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.5, scale=(0.05, 0.2), ratio=(0.5, 1.5), value=0, inplace=False),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [5]:
class RandomTransformDataset:
    def __init__(self, dataset, transform, random_time):
        self.dataset = dataset
        self.transform = transform
        self.random_time = random_time
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        jpg, ans = self.dataset[idx]
        imgs = [self.transform(jpg) for i in range(self.random_time)]
        # (RANDOM_NUM, 3, shape0, shape1)
        return torch.stack(imgs, 0)
    
    def merge_batch(self, imgs, ans=None):
        # imgs: (batch, RANDOM_NUM, 3, shape0, shape1) -> (batch*RANDOM_NUM, ...)
        # ans:  (batch) -> (batch*RANDOM_NUM)
        # if ans:
        #     return imgs.reshape(-1, *imgs.shape[2:]), torch.repeat_interleave(ans, self.random_time)
        # else:
        return imgs.reshape(-1, *imgs.shape[2:])
    
    def merge_predict(self, predicts):
        # (batch*RANOM_NUM, 11)
        res = []
        labels = torch.argmax(predicts, dim=1).squeeze()
        # (batch*RANOM_NUM)
        for prob in torch.split(labels, self.random_time):
            # (RANDOM_NUM)
            # voting
            res.append(torch.argmax(torch.bincount(prob)))
        # (batch)
        return  res

class CLRDataset:
    def __init__(self, dataset, transformA , transformB):
        self.dataset = dataset
        self.transformA = transformA
        self.transformB = transformB
    
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        jpg, ans = self.dataset[idx]
        imgs = [self.transformA(jpg), self.transformB(jpg)]
        # (RANDOM_NUM, 3, shape0, shape1)
        return torch.stack(imgs, 0), ans


In [6]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
data_folder = "/data/ML2021/hw3/"

train_set = DatasetFolder(data_folder+"food-11/training/labeled", loader=lambda x: Image.open(x),extensions="jpg", transform=train_tfm)
# clr_train_set = CLRDataset(train_set, train_tfm, train_tfm)
valid_set = DatasetFolder(data_folder+"food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
# test_set = DatasetFolder(data_folder+"food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

unlabeled_set = DatasetFolder(data_folder+"food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=None)
unlabeled_set.classes = train_set.classes
unlabeled_set.class_to_idx = train_set.class_to_idx
clr_unlab_set = CLRDataset(unlabeled_set, train_tfm, train_tfm)

# Construct data loaders.
# TODO: CLR on labeled data
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True, drop_last=True)
unlab_loader = DataLoader(clr_unlab_set, batch_size=batch_size//2, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [7]:
RANDOM_NUM = 16
test_set = DatasetFolder(data_folder+"food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=None)
rd_test_set = RandomTransformDataset(test_set, train_tfm, random_time=RANDOM_NUM)
rd_test_loader = DataLoader(rd_test_set, batch_size=batch_size//RANDOM_NUM*2, shuffle=False, num_workers=16)
test_loader = DataLoader(DatasetFolder(data_folder+"food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm), batch_size=batch_size, shuffle=False, num_workers=16)


## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [8]:
class BlockConv2d(nn.Module):
    def __init__(self, ch_in, ch_out, k, stride=1,
            act=None, pooling=None, use_bn=True, drop=0, is_dconv=False, is_pad=True):

        super(BlockConv2d, self).__init__()
        pad = k//2 if is_pad else 0
        if is_dconv:
            conv = nn.ConvTranspose2d(ch_in, ch_out, k, padding=pad, stride=stride, output_padding=stride-1)
        else:
            conv = nn.Conv2d(ch_in, ch_out, k, padding=pad, stride=stride)
            
        list = [
            conv
        ]
        if use_bn: list.append(nn.BatchNorm2d(ch_out))
        if act: list.append(act)
        if pooling: list.append(pooling)
        if drop > 0: list.append(nn.Dropout2d(drop))
        self.net = nn.Sequential(*list)

    def forward(self, x):
          return self.net(x)

class BlockLinear(nn.Module):
    def __init__(self, ch_in, ch_out, act=None, use_bn=True, drop=0, bias=True):
        super(BlockLinear, self).__init__()
        list = [
            nn.Linear(ch_in, ch_out, bias=bias)
        ]
        if use_bn: list.append(nn.BatchNorm1d(ch_out))
        if act: list.append(act)
        if drop > 0: list.append(nn.Dropout(drop))
        self.net = nn.Sequential(*list)

    def forward(self, x):
          return self.net(x)

In [9]:
# m = torchvision.models.resnext50_32x4d(pretrained=False)
# for i, j in (m.state_dict().items()):
    # print(i, "\t", j.shape)
# m.fc

In [10]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # input image size: [3, 128, 128]
        self.dim = 2048

        self.encoder = torchvision.models.resnext50_32x4d(pretrained=False, zero_init_residual=True)
        # self.encoder.fc = BlockLinear(2048, self.dim, act=nn.ReLU(), use_bn=True, drop=.5)
        # self.encoder.fc = nn.Identity()
        
        self.act = nn.ReLU()
        self.n = [self.dim, self.dim*2, self.dim*2]
        self.l = len(self.n)
        fc = [
            BlockLinear(self.n[i], self.n[i+1], act=nn.ReLU(), use_bn=True, drop=0, bias=False)
            for i in range(self.l-2)
        ]
        fc += [nn.Linear(self.n[-2], self.n[-1], bias=False)]
        self.encoder.fc = nn.Sequential(*fc)

        self.n2 = [self.n[-1], 1024, 256]
        self.l2 = len(self.n2)
        fc2 = [
            BlockLinear(self.n2[i], self.n2[i+1], act=nn.ReLU(), use_bn=True, drop=0)
            for i in range(self.l2-1)
        ]
        self.classifier = nn.Sequential(*fc2)

        self.out = nn.Linear(self.n2[-1], 11)

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]
        x = self.encoder(x)
        x = self.classifier(x)
        x = self.out(x)
        return x
    
    def predict(self, x):
        """ Predict class by representation """
        return self.out(self.classifier(x))
    
    def repr(self, x):
        """ Encode to the representation """
        return self.encoder(x)
    

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
# Explicitly define the psudolabels in Subset class
def get_pseudosample(self, idx):
    return self.dataset[self.indices[idx]][0], self.labels[idx]

def init_pseudolabel(self, dataset, labels, indices):
    self.dataset = dataset
    self.labels = labels
    self.indices = indices
Subset.__getitem__ = get_pseudosample
Subset.__init__ = init_pseudolabel

In [13]:
def get_pseudo_labels(dataset, model, threshold=0.5):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)
    idx = []
    targets = []
    # Iterate over the dataset by batches.
    for i, batch in tqdm(enumerate(unlab_loader), leave=False, desc='PseudoLabels'):
        img, _ = batch
        with torch.no_grad():
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)
        st = torch.topk(probs, 2, dim=1)
        
        # ---------- TODO ----------
        # Filter the data and construct a new dataset.
        probs1 = st[0][:, 0]
        # probs2 = st[0][:, 1]
        # select = (probs1 > threshold) & ((probs1-probs2) > .25)
        select = (probs1 > threshold)
        targets += st[1][select, 0].tolist()
        idx += (torch.where(select)[0] + batch_size*i).tolist()

    # custom subset
    new = Subset(dataset, targets, idx)
    model.train()
    return new

In [14]:
log_path = 'log'
if log_path in os.listdir():
    os.remove(log_path)
    print('remove log')
log = open(log_path, 'a')

remove log


In [15]:
def train(imgs, is_supervised=False, ans=None, criterion=None, clr_lambda=1):
    imgs = imgs.to(device)
    reprA = model.repr(imgs[:, 0])
    reprB = model.repr(imgs[:, 1])

    reprA_norm = (reprA - reprA.mean(0)) / (reprA.std(0)+1e-7)
    reprB_norm = (reprB - reprB.mean(0)) / (reprB.std(0)+1e-7)

    # covariance matrix
    c = torch.mm(reprA_norm.T, reprB_norm) / len(reprB_norm)
    # loss: offdiag -> 0
    c_offdiag = clr_reg*(torch.triu(c, diagonal=1)**2).sum()
    # loss: diag -> 1
    c_diag = ((c.diag()-1)**2).sum()
    loss = clr_lambda*(c_offdiag+c_diag)
    
    if is_supervised:
        pred = model.predict(torch.cat([reprA, reprB], 0))
        ans = ans.repeat(2)
        # loss = criterion(pred, ans.to(device))
        loss += criterion(pred, ans.to(device))
        return loss, pred, ans, c_diag.item(), c_offdiag.item()
    return loss, c_diag.item(), c_offdiag.item()

In [16]:
model_path = "./model.ckpt"

model = Classifier().to(device)
# model.load_state_dict(torch.load(model_path))

lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, threshold=2e-4, min_lr=1e-5, verbose=True,
)



n_epochs = 200
# classifier | CLR | classifier
n_phase1 = 50
n_phase2 = 100
best_acc = 0
valid_acc = 0

best_semi = 0
semi_flg = False
semi_count = 0
clr_reg = 1e-3
# clr_reg = 2/(model.dim*(model.dim-1))
clr_lambda = 1/1000
loader = train_loader
for epoch in range(n_epochs):
#     if epoch == n_pretrain:
#         optimizer.param_groups[0]['lr'] = lr
#         lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.8, patience=5, threshold=2e-4, min_lr=1e-5, verbose=True
# )

    model.train()

    # ---------- CLR ----------
    # if False:
    if n_phase1 <= epoch < n_phase2:
        train_loss = []
        diag_loss = []
        offdiag_loss = []
        for batch in tqdm(unlab_loader, desc='CLR', leave=False):
            loss, c_diag, c_offdiag = train(batch[0])
            optimizer.zero_grad()
            loss.backward()
            # grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            optimizer.step()
            train_loss.append(loss.item())
            diag_loss.append(c_diag)
            offdiag_loss.append(c_offdiag)

        train_loss = np.mean(train_loss)
        diag_loss = np.mean(diag_loss)
        offdiag_loss = np.mean(offdiag_loss)

        log_str = (f"[ CLR   | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, diag = {diag_loss:5.2f}, offdiag = {offdiag_loss:5.2f}")
        log.write(log_str+'\n')
        log.flush()
        print(log_str)

    # ---------- Training ----------

    if not(n_phase1 <= epoch < n_phase2):
        train_loss = []
        train_accs = []
        # diag_loss = []
        # offdiag_loss = []
        # # for batch in train_loader:
        for batch in tqdm(train_loader, desc='Train', leave=False):

            imgs, labels = batch
            # loss, logits, labels, c_diag, c_offdiag = train(
            #     imgs, 
            #     is_supervised=True, ans=labels, criterion=criterion, clr_lambda=clr_lambda
            # )
            logits = model(imgs.to(device))
            loss = criterion(logits, labels.to(device))
            optimizer.zero_grad()
            loss.backward()
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
            optimizer.step()

            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
            train_loss.append(loss.item())
            train_accs.append(acc)
            # diag_loss.append(c_diag)
            # offdiag_loss.append(c_offdiag)

        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)
        # diag_loss = np.mean(diag_loss)
        # offdiag_loss = np.mean(offdiag_loss)
        # train_loss = 1
        # train_acc = 1

        # log_str = (f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}, diag = {diag_loss:5.2f}, offdiag = {offdiag_loss:5.2f}")
        log_str = (f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
        log.write(log_str+'\n')
        print(log_str)
    
    # ---------- Validation ----------
    model.eval()
    
    valid_loss = []
    valid_accs = []

    # for batch in valid_loader:
    for batch in tqdm(valid_loader, desc='Valid', leave=False):
        imgs, labels = batch

        with torch.no_grad():
          logits = model(imgs.to(device))

        loss = criterion(logits, labels.to(device))
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)
    if valid_acc >= best_acc:
        best_acc = valid_acc
        best_model = model.state_dict()
        torch.save(model.state_dict(), model_path)
        log_str = ('saving model with acc {:.3f}'.format(best_acc))
        print(log_str)
        log.write(log_str+"\n")
        semi_count = 0
    
    # semi early stop
    # if semi_flg:
    #     if valid_acc > best_semi:
    #         best_semi = valid_acc
    #         semi_count = 0
    #     else:
    #         semi_count += 1

    log_str = f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ]  loss = {valid_loss:.5f}, acc = {valid_acc:.5f}"
    log.write(log_str+'\n')
    print(log_str)
    # print(log_str1)
    # log.write(log_str1+"\n")
    log.flush()
    if not(n_phase1 <= epoch < n_phase2):
        lr_scheduler.step(valid_loss)
    # print(f"[ Valid\t| {epoch + 1:03d}/{n_epochs:03d} ] ae_loss = {valid_ae:.5f}, loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

Train:   0%|          | 0/24 [00:00<?, ?it/s]

[ Train | 001/200 ] loss = 2.35270, acc = 0.16569


Valid:   0%|          | 0/6 [00:00<?, ?it/s]

saving model with acc 0.179
[ Valid | 001/200 ]  loss = 2.25937, acc = 0.17943


Train:   0%|          | 0/24 [00:00<?, ?it/s]

...

[ Valid | 199/200 ]  loss = 1.41726, acc = 0.49688


Train:   0%|          | 0/24 [00:00<?, ?it/s]

[ Train | 200/200 ] loss = 1.48912, acc = 0.47233


Valid:   0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 200/200 ]  loss = 1.42137, acc = 0.50391


## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


it = iter(test_loader)
a = next(it)[0].to(device)
ae_model.eval()
b = ae_model(a)
import cv2
b = b.to('cpu').detach().numpy().transpose([0, 2, 3, 1])
a = a.to('cpu').detach().numpy().transpose([0, 2, 3, 1])
def norm(img):
    for i in range(3):
        ch = img[:, :, i]
        a = ch.min()
        b = ch.max()
        img[:, :, i] = (ch-a)/(b-a)
    return (img*255).astype('uint8')
for i, img in enumerate(b):
    cv2.imwrite(f"img/{i}.png", norm(img))
    cv2.imwrite(f"img/{i}_.png", norm(a[i]))
    if i == 0:
        break

In [17]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model_path = './model.ckpt'
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(rd_test_loader):
    # imgs, _ = batch
    imgs = rd_test_set.merge_batch(batch)

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))
    
    # Take the class with greatest logit as prediction and record it.
    # predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
    predictions.extend(rd_test_set.merge_predict(logits))

  0%|          | 0/210 [00:00<?, ?it/s]

In [18]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")